# Classification NBA Model

## Configuration

## Imports

In [20]:
import numpy as np
import pandas as pd

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    confusion_matrix,
    f1_score,
    make_scorer,
    precision_score,
    recall_score,
)
from sklearn.model_selection import KFold, cross_validate, train_test_split
from utils import clean_dataframe_for_training


## Load Data

In [21]:
path = "/home/coder/repos/terminus-eu-complian/data/energy_labels_tabular.csv"

df_stats = pd.read_csv(path)

dtype_dict = {col: str for col in df_stats.columns if "ID" in col.upper()}

df_stats = pd.read_csv(
    path,
    dtype=dtype_dict
)
df_stats['GAME_DATE'] = pd.to_datetime(df_stats['GAME_DATE']).dt.strftime('%Y-%m-%d')

/tmp/ipykernel_1744/3975631190.py:3: DtypeWarning: Columns (497,498,499,540,541,542,583,584,585,626,627,628,669,670,671,712,713,714,1200,1201,1202,1243,1244,1245,1286,1287,1288,1329,1330,1331,1372,1373,1374,1415,1416,1417) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stats = pd.read_csv(path)
/tmp/ipykernel_1744/3975631190.py:7: DtypeWarning: Columns (497,498,499,540,541,542,583,584,585,626,627,628,669,670,671,712,713,714,1200,1201,1202,1243,1244,1245,1286,1287,1288,1329,1330,1331,1372,1373,1374,1415,1416,1417) have mixed types. Specify dtype option on import or set low_memory=False.
  df_stats = pd.read_csv(


In [22]:
# #print column names with name in lower case
# for col in df_stats.columns:
#     if not col.isupper():
#         print(col)

In [23]:
#filter for last two seasons, gett seasons 2025 and 2024
# df_stats = df_stats[df_stats['SEASON_YEAR'].isin([2024, 2025])]

In [24]:
df_stats

,TOTAL_OVER_UNDER_LINE,TEAM_ID_TEAM_HOME,TEAM_CITY_TEAM_HOME,TEAM_ABBREVIATION_TEAM_HOME,TEAM_NAME_TEAM_HOME,MATCHUP_TEAM_HOME,GAME_NUMBER_TEAM_HOME,TEAM_ID_TEAM_AWAY,TEAM_CITY_TEAM_AWAY,TEAM_ABBREVIATION_TEAM_AWAY,...,TRAVEL_RECENCY_RATIO_AWAY_2D_OVER_14D,REST_DAYS_DIFF_HOME_MINUS_AWAY,INJURY_PTS_SHARE_HOME,INJURY_PTS_SHARE_AWAY,STAR_PTS_PCT_DIFF_HOME_MINUS_AWAY,POSS_X_TSPCT_HOME,POSS_X_TSPCT_AWAY,IS_WEEKEND,MONTH,IS_US_HOLIDAY
0,225.0,1610612744,Golden State,GSW,Golden State Warriors,GSW vs. TOR,45,1610612761,Toronto,TOR,...,0.911623,-1,0.342942,0.334240,0.035261,59.681545,56.186177,0,1,0
1,238.5,1610612762,Utah,UTA,Utah Jazz,UTA vs. MIN,44,1610612750,Minnesota,MIN,...,0.000000,-2,0.464479,0.038945,-0.044902,60.155182,61.740477,0,1,0
2,228.0,1610612743,Denver,DEN,Denver Nuggets,DEN vs. LAL,44,1610612747,Los Angeles,LAL,...,0.787565,0,0.434814,0.309755,-0.078416,59.305288,58.197857,0,1,0
3,220.5,1610612745,Houston,HOU,Houston Rockets,HOU vs. SAS,41,1610612759,San Antonio,SAS,...,0.000000,1,0.050331,0.171602,0.010694,56.478101,57.227952,0,1,0
4,238.5,1610612758,Sacramento,SAC,Sacramento Kings,SAC vs. MIA,44,1610612748,Miami,MIA,...,0.900145,1,0.134210,0.209058,-0.022231,55.800122,59.403170,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10865,NaN,1610612763,Memphis,MEM,Memphis Grizzlies,MEM vs. NOP,1,1610612740,New Orleans,NOP,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,10,0
10866,NaN,1610612738,Boston,BOS,Boston Celtics,BOS vs. MIL,2,1610612749,Milwaukee,MIL,...,NaN,1,NaN,NaN,NaN,NaN,NaN,0,10,0
10867,NaN,1610612742,Dallas,DAL,Dallas Mavericks,DAL vs. ATL,1,1610612737,Atlanta,ATL,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,10,0
10868,NaN,1610612739,Cleveland,CLE,Cleveland Cavaliers,CLE vs. BOS,1,1610612738,Boston,BOS,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,10,0


In [25]:

df_to_train = clean_dataframe_for_training(df_stats, nan_threshold=45, drop_all_na_rows=False, verbose=1)


STARTING DATAFRAME CLEANING PIPELINE
Starting basic cleaning with 10870 rows
Basic cleaning complete: 8050 rows remaining

Starting advanced column cleaning with 1805 columns

Advanced column cleaning complete: 1805 → 1141 columns (664 removed)


Dropping NA rows for SEASON_YEAR 2017...
   Removed 0 rows with NaN values from 2017 season

Applying missing data policy...

Missing Data Policy Report:
  Rows dropped: 0 (0.0%)
  Critical columns requiring data: 5
  Columns zero-filled: 132
  Infer pairs applied: 54/228
  Remaining NaN cells: 165866
CLEANING COMPLETE
Final shape: (8050, 1141)


In [27]:
# Count NAs per column
na_counts = df_to_train.isna().sum()

# Get most common SEASON_YEAR for nulls in each column
most_common_season = []
for col in df_to_train.columns:
    if na_counts[col] > 0:
        # Get rows where this column is null
        null_rows = df_stats[df_stats[col].isna()]
        if len(null_rows) > 0 and 'SEASON_YEAR' in df_stats.columns:
            # Find most common SEASON_YEAR for these null rows
            common_season = null_rows['SEASON_YEAR'].mode()
            most_common_season.append(common_season.iloc[0] if len(common_season) > 0 else None)
        else:
            most_common_season.append(None)
    else:
        most_common_season.append(None)

na_counts_df = pd.DataFrame({
    'Column': na_counts.index,
    'NA_Count': na_counts.values,
    'NA_Percentage': (na_counts.values / len(df_to_train) * 100).round(2),
    'Most_Common_Season_Year': most_common_season
}).sort_values('NA_Count', ascending=False)

# Show only columns with NAs
na_counts_df[na_counts_df['NA_Count'] > 0]

,Column,NA_Count,NA_Percentage,Most_Common_Season_Year
938,spread_pct_bets_home,3172,39.40,2017.0
939,spread_pct_money_home,3172,39.40,2017.0
936,total_pct_bets_under,3154,39.18,2017.0
937,total_pct_money_under,3154,39.18,2017.0
940,moneyline_pct_bets_home,3088,38.36,2017.0
...,...,...,...,...
970,spread_fanduel_line_home,4,0.05,2017.0
958,total_fanduel_line_over,3,0.04,2017.0
982,ml_fanduel_price_home,3,0.04,2017.0
959,total_fanduel_price_under,3,0.04,2017.0


In [28]:
df_to_train['LINE_ERROR'] = df_to_train['TOTAL_POINTS'] - df_to_train['TOTAL_OVER_UNDER_LINE']

## Train / Test

In [ ]:
X = df_to_train.drop(['TOTAL_POINTS', 'LINE_ERROR', 'SEASON_YEAR'], axis=1, errors='ignore')
y = df_to_train['LINE_ERROR']

In [31]:
# Train / Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)

In [32]:
df_to_train['IS_TRAINING_DATA'] = False

# Mark True for the rows in the training set
df_to_train.loc[X_train.index, 'IS_TRAINING_DATA'] = True
# output_name = f"{data_path}/training_data_with_missing_data_handled_from_2004-10-01_to_2026-01-10_classifier.csv"
# df_to_train.to_csv(output_name, index=False)

In [33]:
print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
# Check number of coulmns
print(f"Number of columns in training set: {X_train.shape[1]}")
print(f"Number of columns in test set: {X_test.shape[1]}")

Training set size: 6037
Test set size: 2013
Number of columns in training set: 1139
Number of columns in test set: 1139


In [34]:
from tabpfn import TabPFNRegressor
from tabpfn.constants import ModelVersion


# Use it like any sklearn model
model = TabPFNRegressor()
model.fit(X_train, y_train)
# Get predictions
predictions = model.predict(X_test)

In [35]:
from tabpfn.model_loading import (
    load_fitted_tabpfn_model,
    save_fitted_tabpfn_model,
)

save_fitted_tabpfn_model(model, "my_reg.tabpfn_fit")

# Later or on a CPU-only machine
reg_cpu = load_fitted_tabpfn_model("my_reg.tabpfn_fit", device="cpu")

In [39]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# Predict on the test set

def over_under_betting_accuracy(true_error: np.ndarray, pred_error: np.ndarray) -> float:
    true_sign = np.sign(true_error)
    pred_sign = np.sign(pred_error)

    valid = (true_sign != 0) & (pred_sign != 0)
    if not np.any(valid):
        return np.nan

    return float(np.mean(true_sign[valid] == pred_sign[valid]))

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
betting_acc = over_under_betting_accuracy(y_test.values, predictions)

print("Mean Squared Error (MSE):", mse)
print("R² Score:", r2)
print("Mean Absolute Error (MAE):", mae)
print("Over/Under Betting Accuracy:", betting_acc)

Mean Squared Error (MSE): 280.8734744278368
R² Score: 0.04925936466871206
Mean Absolute Error (MAE): 13.25594108847631
Over/Under Betting Accuracy: 0.5476070528967254


In [40]:
import numpy as np
import pandas as pd


def error_sign_accuracy(
    y_true_error,
    y_pred_error,
    include_pushes: bool = False,
) -> float:
    """
    Directional accuracy: sign(true_error) == sign(pred_error).

    Push handling:
    - Default: exclude any sample where either sign == 0
    - If include_pushes=True: count (0,0) as correct, exclude mixed 0/non-0
    """
    y_true_error = np.asarray(y_true_error, dtype=float)
    y_pred_error = np.asarray(y_pred_error, dtype=float)

    true_sign = np.sign(y_true_error)
    pred_sign = np.sign(y_pred_error)

    if include_pushes:
        valid = ~((true_sign == 0) ^ (pred_sign == 0))
    else:
        valid = (true_sign != 0) & (pred_sign != 0)

    if not np.any(valid):
        return np.nan

    return float(np.mean(true_sign[valid] == pred_sign[valid]))

def evaluate_by_confidence_quantiles(
    y_true_error: pd.Series,
    y_pred_error: np.ndarray,
    quantiles=(0.05, 0.10, 0.20, 0.30, 0.50, 1.0),
    include_pushes: bool = False,
) -> pd.DataFrame:
    """
    Directional accuracy on the top-q most confident predictions,
    where confidence = abs(pred_error).
    """
    y_true = y_true_error.to_numpy(dtype=float)
    y_pred = np.asarray(y_pred_error, dtype=float)

    abs_pred = np.abs(y_pred)
    n_total = len(abs_pred)

    order = np.argsort(-abs_pred)

    rows = []
    for q in quantiles:
        k = int(np.ceil(q * n_total))
        idx = order[:k]

        acc = (
            np.nan
            if k == 0
            else error_sign_accuracy(
                y_true[idx],
                y_pred[idx],
                include_pushes=include_pushes,
            )
        )

        rows.append({
            "top_confidence_fraction": float(q),
            "n_games": k,
            "pct_of_test": (k / n_total) if n_total else np.nan,
            "directional_accuracy": acc,
            "min_abs_pred_error": float(abs_pred[idx].min()) if k else np.nan,
            "mean_abs_pred_error": float(abs_pred[idx].mean()) if k else np.nan,
        })

    return pd.DataFrame(rows)

def evaluate_by_abs_pred_error_thresholds(
    y_true_error: pd.Series,
    y_pred_error: np.ndarray,
    thresholds = np.arange(0.0, 11.0, 0.5),
    include_pushes: bool = False,
) -> pd.DataFrame:
    """
    Directional accuracy conditional on abs(pred_error) > t.
    """
    y_true = y_true_error.to_numpy(dtype=float)
    y_pred = np.asarray(y_pred_error, dtype=float)

    abs_pred = np.abs(y_pred)
    n_total = len(y_true)

    rows = []
    for t in thresholds:
        mask = abs_pred > t
        n = int(mask.sum())

        acc = (
            np.nan
            if n == 0
            else error_sign_accuracy(
                y_true[mask],
                y_pred[mask],
                include_pushes=include_pushes,
            )
        )

        rows.append({
            "abs_pred_error_gt": float(t),
            "n_games": n,
            "pct_of_test": (n / n_total) if n_total else np.nan,
            "directional_accuracy": acc,
            "mean_abs_pred_error": float(abs_pred[mask].mean()) if n else np.nan,
        })

    return pd.DataFrame(rows)


In [41]:
# Threshold-based evaluation
thr_df = evaluate_by_abs_pred_error_thresholds(
    y_true_error=y_test,
    y_pred_error=predictions,
    thresholds=np.arange(0.0, 5.0, 0.2),
)

display(
    thr_df.style.format({
        "pct_of_test": "{:.1%}",
        "directional_accuracy": "{:.2%}",
        "mean_abs_pred_error": "{:.2f}",
    })
)

# Quantile-based evaluation
quant_df = evaluate_by_confidence_quantiles(
    y_true_error=y_test,
    y_pred_error=predictions,
)

display(
    quant_df.style.format({
        "pct_of_test": "{:.1%}",
        "directional_accuracy": "{:.2%}",
        "mean_abs_pred_error": "{:.2f}",
    })
)


,abs_pred_error_gt,n_games,pct_of_test,directional_accuracy,mean_abs_pred_error
0,0.000000,2013,100.0%,54.76%,1.22
1,0.200000,1710,84.9%,55.65%,1.42
2,0.400000,1370,68.1%,56.34%,1.70
3,0.600000,1023,50.8%,57.98%,2.11
4,0.800000,743,36.9%,60.54%,2.63
5,1.000000,499,24.8%,63.23%,3.48
6,1.200000,339,16.8%,64.99%,4.61
7,1.400000,226,11.2%,69.64%,6.27
8,1.600000,176,8.7%,73.56%,7.63
9,1.800000,143,7.1%,78.17%,9.00


,top_confidence_fraction,n_games,pct_of_test,directional_accuracy,min_abs_pred_error,mean_abs_pred_error
0,0.050000,101,5.0%,87.00%,2.572486,11.90
1,0.100000,202,10.0%,70.50%,1.462345,6.85
2,0.200000,403,20.0%,64.25%,1.105770,4.06
3,0.300000,604,30.0%,61.10%,0.903488,3.04
4,0.500000,1007,50.0%,58.21%,0.613358,2.13
5,1.000000,2013,100.0%,54.76%,0.000638,1.22
